In [20]:
# Utiliser le code suivant pour récuperer le 
import os
print("JAVA_HOME =", os.environ.get("JAVA_HOME"))

JAVA_HOME = /opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home


In [21]:
import os
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
import findspark
findspark.init()


In [ ]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Creation de my_spark
my_spark = SparkSession.builder.appName("my_spark").getOrCreate()

# Print my_spark
print(my_spark)

# Lecture du fichier csv:
ventes_df = my_spark.read.csv("ventes.csv", header=True, inferSchema=True)

ventes_df.show()


+--------------+-----------+-----------+-------------+-----------------+------------------+---------------+---------------+---------------+-------------+---------------------+-------------------+---------+--------------+
|id_transaction| client_nom| client_age| client_ville| produit_nom     | produit_categorie| produit_marque| prix_catalogue| magasin_nom   | magasin_type| magasin_region      |         date      | quantite| montant_total|
+--------------+-----------+-----------+-------------+-----------------+------------------+---------------+---------------+---------------+-------------+---------------------+-------------------+---------+--------------+
|           1.0| Alice     |       25.0| Paris       | Ordinateur      | Informatique     | Dell          |          800.0| Boutique Lyon | Physique    |  Auvergne-Rhône-A...|2023-03-12 00:00:00|        2|              |
|           2.0| Bob       |       34.0| Lyon        | Smartphone      | Téléphonie       | Apple         |         

['id_transaction',
 ' client_nom',
 ' client_age',
 ' client_ville',
 ' produit_nom     ',
 ' produit_categorie',
 ' produit_marque',
 ' prix_catalogue',
 ' magasin_nom   ',
 ' magasin_type',
 ' magasin_region      ',
 ' date      ',
 ' quantite',
 ' montant_total']

## Nettoyage des données avec PySpark

### <span style=color:orange > Convertir toutes les valeurs de type string en lowercase.

In [27]:
from pyspark.sql.functions import col, trim, lower

string_cols = [colonne for colonne, data_type in ventes_df.dtypes if data_type == 'string']

for c in string_cols:
    ventes_df = ventes_df.withColumn(c, lower(trim(col(c))))

In [ ]:
# Vérification :

ventes_df.show(5)

+--------------+-----------+-----------+-------------+-----------------+------------------+---------------+---------------+---------------+-------------+---------------------+-------------------+---------+--------------+
|id_transaction| client_nom| client_age| client_ville| produit_nom     | produit_categorie| produit_marque| prix_catalogue| magasin_nom   | magasin_type| magasin_region      |         date      | quantite| montant_total|
+--------------+-----------+-----------+-------------+-----------------+------------------+---------------+---------------+---------------+-------------+---------------------+-------------------+---------+--------------+
|           1.0|      alice|       25.0|        paris|       ordinateur|      informatique|           dell|          800.0|  boutique lyon|     physique| auvergne-rhône-alpes|2023-03-12 00:00:00|        2|              |
|           2.0|        bob|       34.0|         lyon|       smartphone|        téléphonie|          apple|         

### <span style=color:orange > Eliminez les lignes où au moins une colonne est manquante

In [ ]:
# Nombre total de lignes avant le traitement:
print(f"Total lignes : {ventes_df.count()}")

Total lignes : 495


In [ ]:
# Liste des colonnes à tester (toutes sauf la dernière)
colonnes = ventes_df.columns[:-1]

# Condition : au moins une colonne est null ou vide
condition = None
for c in colonnes:
    cond = (col(c).isNull()) | (trim(col(c)) == "")
    condition = cond if condition is None else (condition | cond)

# Appliquer le filtre
ventes_df.filter(condition).show(truncate=False)



+--------------+-----------+-----------+-------------+-----------------+------------------+---------------+---------------+---------------+-------------+---------------------+-------------------+---------+--------------+
|id_transaction| client_nom| client_age| client_ville| produit_nom     | produit_categorie| produit_marque| prix_catalogue| magasin_nom   | magasin_type| magasin_region      | date              | quantite| montant_total|
+--------------+-----------+-----------+-------------+-----------------+------------------+---------------+---------------+---------------+-------------+---------------------+-------------------+---------+--------------+
|12.0          |emma       |31.0       |toulouse     |casque audio     |accessoires       |sony           |150.0          |boutique lyon  |physique     |auvergne-rhône-alpes |2023-02-19 00:00:00|         |              |
|58.0          |emma       |31.0       |toulouse     |tablette         |informatique      |samsung        |600.0    